# 8. 프로젝트 - Movielens 영화 추천 실습

## 8-1. 데이터 준비와 전처리

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
#데이터셋 병합. merge 사용.
result = pd.merge(ratings, movies, on = 'movie_id')
result.rename(columns={'count' : 'rating'}, inplace = True)
print(len(result))
result.head(len(result))

836478


,user_id,movie_id,rating,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836473,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary


merge후 rating이 count로 변경되어있어있던것 다시 rename사용하여 변경.

## 8-2 분석

In [7]:
#result에 있는 유니크한 영화 개수
result['movie_id'].nunique()

3628

In [8]:
#result에 있는 유니크한 사용자 수 
result['user_id'].nunique()

6039

In [9]:
result['title'] = result['title'].str.lower()
result.head()

,user_id,movie_id,rating,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),Drama
1,2,1193,5,978298413,one flew over the cuckoo's nest (1975),Drama
2,12,1193,4,978220179,one flew over the cuckoo's nest (1975),Drama
3,15,1193,4,978199279,one flew over the cuckoo's nest (1975),Drama
4,17,1193,5,978158471,one flew over the cuckoo's nest (1975),Drama


In [10]:
#가장 인기 있는 영화 30개(평점순)#가장 인기 있는 영화 30개(평점순)
movie_count_rating = result.groupby(['title'], as_index=False).mean()
movie_count_rating.sort_values(by='rating', ascending=False).head(30)

,title,user_id,movie_id,rating,timestamp
1280,"gate of heavenly peace, the (1995)",1948.666667,787.0,5.000000,9.741198e+08
1186,follow the bitch (1998),2869.000000,1830.0,5.000000,9.724390e+08
1625,identification of a woman (identificazione di ...,5948.000000,1360.0,5.000000,1.016564e+09
2854,schlafes bruder (brother of sleep) (1995),1915.000000,989.0,5.000000,9.746939e+08
782,criminal lovers (les amants criminels) (1999),850.000000,3800.0,5.000000,9.753594e+08
765,country life (1994),1631.000000,687.0,5.000000,9.747256e+08
1206,foreign student (1994),3618.000000,572.0,5.000000,9.671202e+08
1991,lured (1947),5313.000000,3656.0,5.000000,9.609204e+08
246,"baby, the (1973)",46.000000,3280.0,5.000000,9.779795e+08
1871,late bloomers (1996),4682.000000,1553.0,5.000000,9.645669e+08


In [11]:
#가장 인기 있는 영화 30개(리뷰순)
movie_count_review = result.groupby('title')['user_id'].count()
movie_count_review.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

## 8-3 내가 선호하는 영화 5가지 추가

In [12]:
my_favorite = ['toy story (1995)' , 'godfather, the (1972)' ,'matrix, the (1999)' ,'men in black (1997)' ,'terminator 2: judgment day (1991)']

my_playlist = pd.DataFrame({'user_id': ['minkyu']*5, 'title': my_favorite, 'rating':[5]*5})

if not result.isin({'user_id':['minkyu']})['user_id'].any(): 
    result = result.append(my_playlist)                            

result.tail(10)       

,user_id,movie_id,rating,timestamp,title,genre
836473,5851,3607.0,5,9.577566e+08,one little indian (1973),Comedy|Drama|Western
836474,5854,3026.0,4,9.583469e+08,slaughterhouse (1987),Horror
836475,5854,690.0,3,9.577443e+08,"promise, the (versprechen, das) (1994)",Romance
836476,5938,2909.0,4,9.572734e+08,"five wives, three secretaries and me (1998)",Documentary
836477,5948,1360.0,5,1.016564e+09,identification of a woman (identificazione di ...,Drama
0,minkyu,NaN,5,NaN,toy story (1995),NaN
1,minkyu,NaN,5,NaN,"godfather, the (1972)",NaN
2,minkyu,NaN,5,NaN,"matrix, the (1999)",NaN
3,minkyu,NaN,5,NaN,men in black (1997),NaN
4,minkyu,NaN,5,NaN,terminator 2: judgment day (1991),NaN


In [13]:
#사용하지 않는 colmes 제거
result = result.drop(['movie_id', 'timestamp', 'genre'], axis = 1)
result

,user_id,rating,title
0,1,5,one flew over the cuckoo's nest (1975)
1,2,5,one flew over the cuckoo's nest (1975)
2,12,4,one flew over the cuckoo's nest (1975)
3,15,4,one flew over the cuckoo's nest (1975)
4,17,5,one flew over the cuckoo's nest (1975)
...,...,...,...
0,minkyu,5,toy story (1995)
1,minkyu,5,"godfather, the (1972)"
2,minkyu,5,"matrix, the (1999)"
3,minkyu,5,men in black (1997)


In [14]:
# 고유한 유저, title를 찾아내는 코드
user_unique = result['user_id'].unique()
title_unique = result['title'].unique()
print(user_unique)
# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['minkyu'])    
print(title_to_idx['godfather, the (1972)'])

[1 2 12 ... 2908 4486 'minkyu']
6039
607


In [15]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_result = result['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_result) == len(result):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    result['user_id'] = temp_user_result   # result['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = result['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(result):
    print('title column indexing OK!!')
    result['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

result

user_id column indexing OK!!
title column indexing OK!!


,user_id,rating,title
0,0,5,0
1,1,5,0
2,2,4,0
3,3,4,0
4,4,5,0
...,...,...,...
0,6039,5,40
1,6039,5,607
2,6039,5,124
3,6039,5,175


## 8-4 CSR matrix 생성

In [16]:

from scipy.sparse import csr_matrix

num_user = result['user_id'].nunique()
num_title = result['title'].nunique()

csr_data = csr_matrix((result.rating, (result.user_id, result.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 8-5 als_model = AlternatingLeastSquares 모델 구성

In [17]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [18]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [19]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [20]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 8-6 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악

In [21]:
minkyu, godfather = user_to_idx['minkyu'], title_to_idx['godfather, the (1972)']
minkyu_vector, godfather_vector = als_model.user_factors[minkyu], als_model.item_factors[godfather]

print('슝=3')

슝=3


In [22]:
minkyu_vector

array([ 2.0685951e-01,  5.7151777e-01, -4.1492127e-02,  2.1467155e-01,
       -1.6004997e+00, -1.1932243e-01, -3.5764050e-01,  1.6949880e-01,
        2.6762530e-01, -7.0355737e-01,  7.0526731e-01, -1.9818401e-01,
        1.0293564e-01, -6.3844478e-01, -8.1441991e-02, -4.6499673e-01,
       -1.3244470e+00, -3.5238969e-01,  4.7078347e-01, -8.8748723e-02,
       -2.7214196e-01,  1.6939494e-01,  2.9031083e-01, -2.8537565e-01,
        1.3326897e-01,  8.9970112e-01, -1.3951119e+00,  4.7367731e-01,
        8.6206359e-01, -4.4016877e-01, -3.0346379e-01, -1.3030876e-01,
       -2.0228902e-01,  3.9448470e-01,  3.4442112e-01,  9.7301871e-02,
       -5.1495272e-01,  1.0674758e+00,  9.7385764e-02, -4.2596701e-01,
        4.0323061e-01, -1.1924384e-03,  5.9037381e-01,  2.1853156e-01,
        3.8087320e-02, -3.9635729e-02, -1.0481019e+00, -8.0213524e-02,
        7.8450596e-01,  4.0375727e-01, -1.5512891e-01,  1.5266125e-01,
        7.1002579e-01,  3.4503579e-01, -2.9819006e-01,  3.8881388e-01,
      

In [23]:
godfather_vector

array([-0.00435666,  0.03454647,  0.02205759,  0.00143888, -0.0327148 ,
        0.01278519, -0.04852063, -0.03917094,  0.01825921, -0.04344388,
        0.04222743, -0.00303288,  0.00116414,  0.00608609, -0.01670842,
       -0.00075222, -0.02118283,  0.00802186,  0.05621288, -0.0308126 ,
       -0.03111573, -0.01176697,  0.02594854, -0.00529061,  0.02584517,
        0.02450242, -0.02367128,  0.00083134,  0.00781164, -0.01707795,
       -0.00360259, -0.01174824, -0.00116812,  0.00017619,  0.00860748,
        0.03422222,  0.01085102,  0.02344715, -0.02466628, -0.01922139,
        0.02509245,  0.01145204,  0.02833307,  0.03020274,  0.01053719,
        0.01863508, -0.04681677,  0.00213889,  0.03628739, -0.01871974,
       -0.01779474,  0.02279111, -0.007868  , -0.00261525, -0.020718  ,
        0.03086005, -0.01343683,  0.03414023, -0.01373793,  0.01543003,
        0.02031061, -0.00730346,  0.05163434,  0.01760673,  0.00519456,
        0.00912949,  0.02288951,  0.01352312, -0.00053897,  0.05

In [24]:
np.dot(minkyu_vector, godfather_vector)

0.6301895

0.63 나름 준수하게 나왔다.

In [25]:
song_of_freedom = title_to_idx['song of freedom (1936)']
song_of_freedom_vector = als_model.item_factors[song_of_freedom]
np.dot(minkyu_vector, song_of_freedom_vector)

-0.00067026156

전혀 관계 없는 영화로 음수값이 나왔다.

## 8-7 내가 좋아하는 영화와 비슷한 영화를 추천받기

In [26]:
favorite_title = 'godfather, the (1972)'
title_id = title_to_idx[favorite_title]
similar_title = als_model.similar_items(title_id, N=15)
similar_title

[(607, 1.0),
 (380, 0.9656974),
 (435, 0.550381),
 (128, 0.44989723),
 (3517, 0.36162058),
 (432, 0.35818225),
 (3150, 0.34919882),
 (601, 0.3427231),
 (269, 0.3328229),
 (3499, 0.32851177),
 (3601, 0.3237231),
 (3531, 0.3120346),
 (361, 0.28345513),
 (2560, 0.28048357),
 (3384, 0.27958035)]

In [27]:
#title_to_idx 를 뒤집어, index로부터 title 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_title]

['godfather, the (1972)',
 'godfather: part ii, the (1974)',
 'godfather: part iii, the (1990)',
 'french connection, the (1971)',
 'paralyzing fear: the story of polio in america, a (1998)',
 'taxi driver (1976)',
 'boys, the (1997)',
 'citizen kane (1941)',
 'goodfellas (1990)',
 'simon sez (1999)',
 'condition red (1995)',
 'aiqing wansui (1994)',
 'casablanca (1942)',
 'ciao, professore! (io speriamo che me la cavo ) (1993)',
 "squanto: a warrior's tale (1994)"]

In [28]:
def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

print("슝=3")

슝=3


In [29]:
get_similar_title('toy story (1995)')

['toy story (1995)',
 'toy story 2 (1999)',
 'aladdin (1992)',
 'babe (1995)',
 "bug's life, a (1998)",
 'groundhog day (1993)',
 'lion king, the (1994)',
 'pleasantville (1998)',
 'beauty and the beast (1991)',
 "there's something about mary (1998)"]

## 8-8 내가 좋아할만한 영화 추천

In [30]:
user = user_to_idx['minkyu']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
title_recommended

[(380, 0.56870806),
 (107, 0.56451386),
 (62, 0.50204515),
 (50, 0.38360828),
 (200, 0.3526919),
 (435, 0.3389294),
 (141, 0.33739114),
 (4, 0.28814802),
 (87, 0.2737376),
 (145, 0.26739258),
 (110, 0.25949684),
 (375, 0.25770527),
 (372, 0.25163338),
 (48, 0.24235785),
 (317, 0.2407107),
 (64, 0.23772332),
 (44, 0.23268099),
 (33, 0.232135),
 (75, 0.22194521),
 (284, 0.21995308)]

In [31]:
[idx_to_title[i[0]] for i in title_recommended]

['godfather: part ii, the (1974)',
 'jurassic park (1993)',
 'total recall (1990)',
 'toy story 2 (1999)',
 'terminator, the (1984)',
 'godfather: part iii, the (1990)',
 'fugitive, the (1993)',
 "bug's life, a (1998)",
 'braveheart (1995)',
 'fifth element, the (1997)',
 'groundhog day (1993)',
 'face/off (1997)',
 'x-men (2000)',
 'saving private ryan (1998)',
 'twelve monkeys (1995)',
 'star wars: episode vi - return of the jedi (1983)',
 'star wars: episode iv - a new hope (1977)',
 'aladdin (1992)',
 'hunt for red october, the (1990)',
 'nightmare before christmas, the (1993)']

In [32]:
#위 목록중 forrest_gump가 설출되는데 끼친 영향
forrest_gump = title_to_idx['forrest gump (1994)']
explain = als_model.explain(user, csr_data, itemid=forrest_gump)

In [33]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('toy story (1995)', 0.0943855072417041),
 ('men in black (1997)', 0.0761094600944155),
 ('matrix, the (1999)', 0.007960673780983),
 ('terminator 2: judgment day (1991)', -0.014409643221498448),
 ('godfather, the (1972)', -0.014762835876905196)]

toy story가 그나마 높은 기여

# 느낀점
데이터 전처리과정에서 가장 많은 시간을 소요했고 그 이후 모델형성부터는 큰 문제 없이 진행되었다.
